# Animal Shelter Outcomes

## Import Libraries: OS, Numpy, Pandas, and Matplotlib

In [301]:
import os
import math
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Set Dataframe Display Configuration

In [35]:
pd.set_option('display.max_rows', 20)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

## Set Matplotlib Display Configuration

In [3]:
%matplotlib inline
plt.style.use('ggplot')

## Read in Dataset

In [224]:
df = pd.read_csv(os.path.join('..', 'datasets', 'shelter-animal-outcomes.csv.gz'), index_col='AnimalID')

## First Look at Dataset

In [50]:
df.shape

(26729, 10)

In [51]:
df.columns

Index([u'AnimalID', u'Name', u'DateTime', u'OutcomeType', u'OutcomeSubtype',
       u'AnimalType', u'SexuponOutcome', u'AgeuponOutcome', u'Breed',
       u'Color'],
      dtype='object')

In [52]:
df.head()

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
1,A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
2,A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White
3,A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream
4,A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan


In [53]:
df.describe()

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
count,26729,19038,26729,26729,13117,26729,26728,26711,26729,26729
unique,26729,6374,22918,5,16,2,5,44,1380,366
top,A705677,Max,2015-08-11 00:00:00,Adoption,Partner,Dog,Neutered Male,1 year,Domestic Shorthair Mix,Black/White
freq,1,136,19,10769,7816,15595,9779,3969,8810,2824


In [106]:
df.DateTime = pd.to_datetime(df.DateTime)

In [109]:
type(df.DateTime[1])

pandas.tslib.Timestamp

## Determine Response(?) Variable

We're interested in determining which (column) is most predictive of the 'Adoption' outcome.

In [113]:
df.OutcomeType.value_counts()

Adoption           10769
Transfer            9422
Return_to_owner     4786
Euthanasia          1555
Died                 197
Name: OutcomeType, dtype: int64

In [115]:
df.OutcomeSubtype.value_counts()

Partner                7816
Foster                 1800
SCRP                   1599
Suffering              1002
Aggressive              320
Offsite                 165
In Kennel               114
Behavior                 86
Rabies Risk              74
Medical                  66
In Foster                52
Enroute                   8
Court/Investigation       6
At Vet                    4
In Surgery                3
Barn                      2
Name: OutcomeSubtype, dtype: int64

In [118]:
pd.crosstab(df.OutcomeSubtype, df.OutcomeType)

OutcomeType,Adoption,Died,Euthanasia,Transfer
OutcomeSubtype,,,,
Aggressive,0,0,320,0
At Vet,0,4,0,0
Barn,1,0,0,1
Behavior,0,0,86,0
Court/Investigation,0,0,6,0
Enroute,0,8,0,0
Foster,1800,0,0,0
In Foster,0,52,0,0
In Kennel,0,114,0,0


In [153]:
df[(df.AnimalType == 'Dog') & ((df.SexuponOutcome == 'Neutered Male') | (df.SexuponOutcome == 'Intact Male'))].Name.value_counts()

Max           112
Buddy          78
Rocky          76
Charlie        75
Chico          52
Jack           50
Duke           43
Lucky          38
Toby           38
Blue           36
             ... 
Cato            1
Beagle          1
Nike            1
G Money         1
Sir Winsor      1
Prada           1
Eubie           1
Yoakum          1
Frisbee         1
Leeland         1
Name: Name, dtype: int64

In [228]:
males    = (df.SexuponOutcome == 'Neutered Male') | (df.SexuponOutcome == 'Intact Male')
females  = (df.SexuponOutcome == 'Spayed Female') | (df.SexuponOutcome == 'Intact Female')
intact   = (df.SexuponOutcome == 'Intact Female') | (df.SexuponOutcome == 'Intact Female')
neutered = (df.SexuponOutcome == 'Neutered Male') | (df.SexuponOutcome == 'Neutered Female')

df_copy = df.copy()

df_copy.loc[males, 'Sex'] = 'Male'
df_copy.loc[females, 'Sex'] = 'Female'
df_copy.loc[intact, 'Steralized'] = False
df_copy.loc[neutered, 'Steralized'] = True
df_copy.drop(['SexuponOutcome'], axis = 1, inplace = True)
df_copy

,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,AgeuponOutcome,Breed,Color,Sex,Steralized
AnimalID,,,,,,,,,,
A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,1 year,Shetland Sheepdog Mix,Brown/White,Male,True
A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,1 year,Domestic Shorthair Mix,Cream Tabby,Female,NaN
A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,2 years,Pit Bull Mix,Blue/White,Male,True
A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,Cat,3 weeks,Domestic Shorthair Mix,Blue Cream,Male,NaN
A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,Dog,2 years,Lhasa Apso/Miniature Poodle,Tan,Male,True
A677334,Elsa,2014-04-25 13:04:00,Transfer,Partner,Dog,1 month,Cairn Terrier/Chihuahua Shorthair,Black/Tan,Female,False
A699218,Jimmy,2015-03-28 13:11:00,Transfer,Partner,Cat,3 weeks,Domestic Shorthair Mix,Blue Tabby,Male,NaN
A701489,NaN,2015-04-30 17:02:00,Transfer,Partner,Cat,3 weeks,Domestic Shorthair Mix,Brown Tabby,NaN,NaN
A671784,Lucy,2014-02-04 17:17:00,Adoption,NaN,Dog,5 months,American Pit Bull Terrier Mix,Red/White,Female,NaN


In [227]:
def my_outcome(outcome):
    if outcome == 'Return_to_owner':
        return 'ReturnToOwner'
    else:
        return outcome
    
df_copy2 = df.copy()
    
df_copy2['OutcomeType2'] = df.OutcomeType.apply(my_outcome)
df_copy2['OutcomeType3'] = df.OutcomeType.apply(lambda outcome: 'ReturnToOwner' if outcome == 'Return_to_owner' else outcome)
df_copy2

,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,...,AgeuponOutcome,Breed,Color,OutcomeType2,OutcomeType3
AnimalID,,,,,,,,,,,
A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,...,1 year,Shetland Sheepdog Mix,Brown/White,ReturnToOwner,ReturnToOwner
A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,...,1 year,Domestic Shorthair Mix,Cream Tabby,Euthanasia,Euthanasia
A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,...,2 years,Pit Bull Mix,Blue/White,Adoption,Adoption
A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,Cat,...,3 weeks,Domestic Shorthair Mix,Blue Cream,Transfer,Transfer
A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,Dog,...,2 years,Lhasa Apso/Miniature Poodle,Tan,Transfer,Transfer
A677334,Elsa,2014-04-25 13:04:00,Transfer,Partner,Dog,...,1 month,Cairn Terrier/Chihuahua Shorthair,Black/Tan,Transfer,Transfer
A699218,Jimmy,2015-03-28 13:11:00,Transfer,Partner,Cat,...,3 weeks,Domestic Shorthair Mix,Blue Tabby,Transfer,Transfer
A701489,NaN,2015-04-30 17:02:00,Transfer,Partner,Cat,...,3 weeks,Domestic Shorthair Mix,Brown Tabby,Transfer,Transfer
A671784,Lucy,2014-02-04 17:17:00,Adoption,NaN,Dog,...,5 months,American Pit Bull Terrier Mix,Red/White,Adoption,Adoption


In [297]:
def split_timeframe(timeframe):
    # pd.is_null()
    try:
        (number, unit) = timeframe.split(' ')
        
        if 'day' in unit:
            return float(number)
        elif 'week' in unit:
            return float(number) * 7
        elif 'month' in unit:
            return float(number) * 30.5
        elif 'year' in unit:
            return float(number) * 365
        else:
            return "NO MATCH!"
    except:
        return timeframe
    
df.AgeuponOutcome.apply(split_timeframe)

AnimalID
A671945     365.0
A656520     365.0
A686464     730.0
A683430      21.0
A667013     730.0
A677334      30.5
A699218      21.0
A701489      21.0
A671784     152.5
A677747     365.0
            ...  
A707871     730.0
A704405      61.0
A662019    1825.0
A687056     152.5
A716997    2190.0
A702446      30.5
A718934      91.5
A698128    1460.0
A677478      28.0
A706629     365.0
Name: AgeuponOutcome, dtype: float64

In [254]:
def is_nan(x):
    if x:
        return True
    else: 
        return False
is_nan(df.OutcomeSubtype[0])

True

In [255]:
df.AgeuponOutcome.unique()

array(['1 year', '2 years', '3 weeks', '1 month', '5 months', '4 years',
       '3 months', '2 weeks', '2 months', '10 months', '6 months',
       '5 years', '7 years', '3 years', '4 months', '12 years', '9 years',
       '6 years', '1 weeks', '11 years', '4 weeks', '7 months', '8 years',
       '11 months', '4 days', '9 months', '8 months', '15 years',
       '10 years', '1 week', '0 years', '14 years', '3 days', '6 days',
       '5 days', '5 weeks', '2 days', '16 years', '1 day', '13 years', nan,
       '17 years', '18 years', '19 years', '20 years'], dtype=object)

In [291]:
"HelloThere".translate(None, 'aeiou')

'HllThr'

In [321]:
def pure_breed(breed):
    if '/' in breed:
        return False
    if re.search(r'^[A-Za-z ]+ Mix', breed):
        return False
    else:
        return True

df[ df.Breed.apply(pure_breed) ]

,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
AnimalID,,,,,,,,,
A677747,NaN,2014-05-03 07:48:00,Adoption,Offsite,Dog,Spayed Female,1 year,Cairn Terrier,White
A686716,Chester,2014-08-27 07:50:00,Return_to_owner,NaN,Dog,Neutered Male,12 years,Beagle,Tricolor
A711392,Molly,2015-09-08 15:31:00,Transfer,Partner,Dog,Intact Female,2 months,English Foxhound,Buff
A681410,Nym,2014-06-22 17:22:00,Adoption,NaN,Cat,Spayed Female,3 months,Domestic Longhair,Brown Tabby
A700740,Charlie,2015-12-17 12:10:00,Adoption,Foster,Dog,Neutered Male,1 year,Chihuahua Shorthair,White
A676200,Georgia,2014-04-10 11:49:00,Transfer,Partner,Dog,Spayed Female,7 years,Lhasa Apso,White/Black
A674530,Jackson,2014-09-11 17:03:00,Adoption,NaN,Dog,Neutered Male,5 years,Pit Bull,Black
A696350,Minnie,2015-12-15 17:30:00,Return_to_owner,NaN,Dog,Spayed Female,1 year,Pit Bull,Brown Brindle
A652097,Minnie,2014-02-10 12:34:00,Return_to_owner,NaN,Dog,Spayed Female,2 years,Dachshund Longhair,Brown
